***本项目是用来演示使用中文菜谱数据调优ChatGLM2-6B***

In [1]:
# 测试的菜名
test_query = "蕉牛奶西米捞"

## 1.预训练模型能力

In [2]:
from transformers import  AutoModel,AutoTokenizer, BitsAndBytesConfig
import torch
torch.cuda.empty_cache()

In [3]:
model_name = "./models/chatglm2-6b" #  #或者远程 “THUDM/chatglm2-6b”
# .cache/huggingface/hub/models--THUDM--chatglm2-6b/snapshots/4e38bef4c028beafc8fb1837462f74c02e68fcc2/

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True, #QLoRA 设计的 Double Quantization
    bnb_4bit_quant_type='nf4', # QLoRA 设计的 Normal Float 4 量化数据类型
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_base = AutoModel.from_pretrained(model_name,
                                  quantization_config=bnb_config,
                                  trust_remote_code=True) #.half().cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
# 使用 Markdown 格式打印模型输出
from IPython.display import display, Markdown, clear_output

In [5]:
# 测试base的能力
def display_answer_base(query, history=[]):
    for response, history in model_base.stream_chat(
            tokenizer, query, history=history):
        clear_output(wait=True)
        display(Markdown(response))
    return history

In [6]:
h = display_answer_base(test_query,[])

蕉牛奶西米捞是一种传统的华南小吃,主要流行于广东、福建、海南等地区。主要由西米、香蕉、牛奶和糖等食材烹制而成。

烹制方法是先将香蕉切成小块,加入锅中慢慢煮至软烂。然后加入西米和牛奶,搅拌均匀。最后加入适量的糖,继续煮至糖完全溶解即可。

蕉牛奶西米捞口感香甜可口,营养丰富,是很多人喜爱的美食之一。

In [7]:
## 1 epoch能力

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
from transformers import  AutoModel,AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

model_name = "./models/chatglm2-6b" #"../chatglm2-6b" #或者远程 “THUDM/chatglm2-6b”
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_a = AutoModel.from_pretrained(model_name,
                                  load_in_8bit=False,
                                  trust_remote_code=True,
                                  device_map='auto')
ckpt_path_a = 'meishi_chatglm2_qlora_1epoch'
model_a = PeftModel.from_pretrained(model_a, ckpt_path_a)
model_a = model_a.merge_and_unload() # 合并QLoRA的权重

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
# 1 epoch能力
def display_answer_1epoch(query, history=[]):
    for response, history in model_a.stream_chat(
            tokenizer, query, history=history):
        clear_output(wait=True)
        display(Markdown(response))
    return history

In [12]:
h = display_answer_1epoch(test_query,[])

食材明细: 
香蕉 : 1根 
西米 : 适量 
牛奶 : 适量 

制作步骤: 
第1步：香蕉去皮切小块，放入搅拌机打成泥。第2步：将香蕉泥倒入小锅中，加入牛奶，搅拌均匀。第3步：倒入小锅中，用中小火煮至沸腾。第4步：将煮好的香蕉牛奶倒入容器中，放凉后加入西米。第5步：用保鲜膜盖住容器，放入冰箱冷藏2小时。第6步：第二天早晨，取出香蕉牛奶，放入蒸锅中蒸15分钟即可。
制作方法: 
煮十分钟

In [13]:
import torch
torch.cuda.empty_cache()

In [14]:
from transformers import  AutoModel,AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

model_name = "./models/chatglm2-6b" #"../chatglm2-6b" #或者远程 “THUDM/chatglm2-6b”
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_b = AutoModel.from_pretrained(model_name,
                                  load_in_8bit=False,
                                  trust_remote_code=True,
                                  device_map='auto')
ckpt_path_b = 'meishi_chatglm2_qlora_3epoch'
model_b = PeftModel.from_pretrained(model_b, ckpt_path_b)
model_b = model_b.merge_and_unload() # 合并QLoRA的权重

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
# 测试3 epoch的能力
# 1 epoch能力
def display_answer_3epoch(query, history=[]):
    for response, history in model_a.stream_chat(
            tokenizer, query, history=history):
        clear_output(wait=True)
        display(Markdown(response))
    return history

In [18]:
h = display_answer_3epoch(test_query,[])

食材明细: 
香蕉 : 适量 
西米 : 适量 
牛奶 : 适量 

制作步骤: 
第1步：香蕉剥皮切小块第2步：西米洗净浸泡20分钟第3步：香蕉切小块第4步：把香蕉和牛奶一起放入锅中第5步：加适量水煮第6步：煮至香蕉和牛奶一起变稠第7步：放西米一起煮至西米粘稠第8步：放冰箱冷藏一下，就可以享用了
制作方法: 
煮十分钟

In [19]:
integrate = "牛肉片 : 300克 \n青椒片 : 75克 \n洋葱片 : 50克 \n沙茶酱 : 75克 \n白糖 : 20克 \n白酱油 : 20克 \n淀粉 : 30克 \n蒜末 : 10克 \n花生酱 : 35克 \n味精 : 3克 \n熟猪油 : 适量 \n"
step = "第1步：牛肉片用白酱油抓匀，再加淀粉拌匀。第2步：将牛肉片加入沙茶酱、花生酱、白糖、蒜末、味精、再放入青椒片、洋葱片淋上熟猪油拌匀，用高档火打6分钟即可。"
method = "炒廿分钟"
prompt = "" + '\n' + '食材明细: \n' + integrate + '\n' + "制作步骤: \n" + step + '\n' + "制作方法: \n" + method + '\n'


In [26]:
h = display_answer_1epoch(prompt,[])

以上步骤是菜品【牛肉沙茶】的制作方法